In [1]:
import tensorflow as tf
import pandas as pd 
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_df = pd.read_csv('Dataset/train.csv')
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
feat_cols = train_df.drop('label',axis=1)

In [4]:
labels = train_df['label']

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
test_df = pd.read_csv('Dataset/test.csv')

In [7]:
#INIT WEIGHTS
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [8]:
#INIT BIAS
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)


In [9]:
#CONV2D

def conv2d(x,W):
    # x --> input tensor --> [batch,H,W,Channels]
    # W --> [filter H, filter W, Channels IN, Channels OUT]

    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')



In [10]:
#POOLING

def max_pool_2by2(x):
    # x --> input tensor --> [batch,H,W,Channels]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


In [11]:
# CONVOLUTIONAL LAYER

def convolutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

# NORMAL (FULLY CONNECTED)

def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W) + b

In [12]:
# PLACEHOLDERS

x = tf.placeholder(tf.float32,shape = [None,784])

y_true = tf.placeholder(tf.float32,shape=[None,10]) #AS one hot encoded

# LAYERS
# RESHAPE X TO 28 x 28. 1 is grayscale.
x_image = tf.reshape(x, [-1,28,28,1])
# 5 x 5 is patch size. 32 is no. of output channels (features computing)
convo_1 = convolutional_layer(x_image,shape=[5,5,1,32])

convo_1_pooling = max_pool_2by2(convo_1)

convo_2 = convolutional_layer(convo_1_pooling,shape = [5,5,32,64])

convo_2_pooling = max_pool_2by2(convo_2)

# 7 by 7 image times output i.e 64

convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])

# 1024 neurons

full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [13]:
# DROPOUT

hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

y_pred = normal_full_layer(full_one_dropout,10)

# LOSS FUNCTION

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

# OPTIMIZER

optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)


In [14]:
test_df = pd.read_csv('Dataset/test.csv')

# output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })
# output.to_csv('titanic-predictions.csv', index = False)
# output.head()

In [15]:
numpyMatrix = test_df.as_matrix()

In [16]:
numpyMatrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [17]:
numpyMatrix_norm = numpyMatrix / 255

In [34]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
steps = 5000

with tf.Session() as sess:
    sess.run(init)

    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y, hold_prob:0.5})
    
        if i%100 == 0:
             print("ON STEP: {}".format(i))
    saver.save(sess,'model/my_model2.ckpt')
#             print("ACCURACY: ")
#             matches = tf.equal(tf.argmax(y_pred,axis=1), tf.argmax(y_true,1))

#             acc = tf.reduce_mean(tf.cast(matches, tf.float32))

#             print(sess.run(acc,feed_dict={x:mnist.test.images, y_true:mnist.test.labels,hold_prob:1.0}))
#             print('\n')

ON STEP: 0
ON STEP: 100
ON STEP: 200
ON STEP: 300
ON STEP: 400
ON STEP: 500
ON STEP: 600
ON STEP: 700
ON STEP: 800
ON STEP: 900
ON STEP: 1000
ON STEP: 1100
ON STEP: 1200
ON STEP: 1300
ON STEP: 1400
ON STEP: 1500
ON STEP: 1600
ON STEP: 1700
ON STEP: 1800
ON STEP: 1900
ON STEP: 2000
ON STEP: 2100
ON STEP: 2200
ON STEP: 2300
ON STEP: 2400
ON STEP: 2500
ON STEP: 2600
ON STEP: 2700
ON STEP: 2800
ON STEP: 2900
ON STEP: 3000
ON STEP: 3100
ON STEP: 3200
ON STEP: 3300
ON STEP: 3400
ON STEP: 3500
ON STEP: 3600
ON STEP: 3700
ON STEP: 3800
ON STEP: 3900
ON STEP: 4000
ON STEP: 4100
ON STEP: 4200
ON STEP: 4300
ON STEP: 4400
ON STEP: 4500
ON STEP: 4600
ON STEP: 4700
ON STEP: 4800
ON STEP: 4900


In [32]:
a = np.split(ary)
a.shape

SyntaxError: invalid syntax (<ipython-input-32-fdf487a9de7c>, line 1)

In [24]:
mnist.test.images.shape

(10000, 784)

In [39]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
steps = 1

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess,'model/my_model.ckpt')

    for i in range(steps):
        print('Currently on step {}'.format(i))
        print('Accuracy is:')
        # Test the Train Model
        matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

        acc = y_pred.reshape([28,28])

        print(sess.run(acc,feed_dict={x:numpyMatrix_norm[:5],y_true:mnist.test.labels,hold_prob:1.0}))
        print('\n')

INFO:tensorflow:Restoring parameters from model/my_model.ckpt
Currently on step 0
Accuracy is:


NameError: name 'y_pre' is not defined